# Hashcode 2019

In [ ]:
import os
import itertools
import multiprocessing
from collections import namedtuple
from operator import itemgetter

In [ ]:
INPUT = "data"
OUTPUT = "solutions"

Parse input

In [ ]:
Image = namedtuple('Image', 'id orientation num_tags tags')
def parse_input(f):
    data = open(f).read().splitlines()
    images = []
    for i, d in enumerate(data[1:]):
        d_split = d.split()
        images.append(Image(i, d_split[0], int(d_split[1]), set(d_split[2:])))
    return images

Helpers for creating slides

In [ ]:
Slide = namedtuple('Slide', 'images num_tags tags')
def gen_slide(s):
    if type(s) == list:
        tags = s[0].tags.union(s[1].tags)
        return Slide([s[0].id, s[1].id], len(tags), tags)
    else:
        return Slide(s.id, s.num_tags, s.tags)

In [ ]:
def sort_images_desc(images, reverse=True):
    return sorted(images, key=itemgetter(Image._fields.index('num_tags')), reverse=reverse)

def sort_slideshow_asc(slideshow, reverse=False):
    return sorted(slideshow, key=itemgetter(Slide._fields.index('num_tags')), reverse=reverse)

Combine v images to build v slides.

1) baseline
2) greedy

In [ ]:
def compute_v_slides(images):
    images = sort_images_desc(images)
    slideshow = []
    i = 0
    while i < len(images) / 2:
        slideshow.append([images[i], images[len(images)-1-i]])
        i += 1
    return slideshow

In [ ]:
def score_v_slide(a, b):
    return len(a.tags.union(b.tags))

In [ ]:
def compute_v_slides_greedy(images, pad=100000):
    """
    Create v slides by combining two v images that yield a slide with
    a high number of unique tags.    
    """
    images = sort_images_desc(images, reverse=True)
    slideshow = []
    while len(images) > 1:
        a = images[0]
        del images[0]
        best_id = len(images) - 1
        best_score = -1
        for i in range(max(0, len(images)-1-pad), len(images) - 1):
            cur_score = score_v_slide(a, images[i])
            if cur_score > best_score:
                best_id = i
                best_score = cur_score
        slideshow.append([a, images[best_id]])
        del images[best_id]
    return slideshow

Reorder slide show (greedy)

In [ ]:
def score(a, b):
    return min(map(len, [a.tags - b.tags, a.tags.intersection(b.tags), b.tags - a.tags]))

In [ ]:
def reorder_slides(slideshow, pad=100000):
    orig_length = len(slideshow)
    slides_sorted = sort_slideshow_asc(slideshow)
    slideshow = [slides_sorted[0]]
    best_id = 0
    del slides_sorted[0]
    while len(slides_sorted) > 0:
        best_score = -1000000
        for i in range(max(0, best_id - pad), min(best_id + pad, len(slides_sorted))):
            cur_score = score(slideshow[-1], slides_sorted[i])
            if cur_score > best_score:
                best_score = cur_score
                best_id = i
        slideshow.append(slides_sorted[best_id])
        del slides_sorted[best_id]
        if len(slideshow) % 1000 == 0:
            print(f"{len(slideshow) / orig_length * 100:2.3f}%")
    return slideshow

In [ ]:
def compute(images):
    v_images = [x for x in images if x.orientation == "V"]
    h_images = [x for x in images if x.orientation == "H"]
    slideshow = h_images
    #slideshow += compute_v_slides(v_images)
    slideshow += compute_v_slides_greedy(v_images)
    slideshow = list(map(gen_slide, slideshow))
    slideshow = reorder_slides(slideshow)
    return slideshow

Generate output

In [ ]:
def generate_output(s):
    result = []
    result.append(f"{len(s)}")
    for slide in s:
        if type(slide.images) == list:
            result.append(f"{slide.images[0]} {slide.images[1]}")
        else:
            result.append(f"{slide.images}")
    return result

---

#### Compute solutions

In [ ]:
def run(i):
    print(i)
    parsed_input = parse_input(os.path.join(INPUT, i))
    s = compute(parsed_input)
    with open(os.path.join(OUTPUT, f"{i}_result"), "w") as f:
        f.writelines(map(lambda x: f"{x}\n", generate_output(s)))

files = sorted([x for x in os.listdir(INPUT) if not x.startswith(".")])
print(files)
#list(map(run, files))
pool = multiprocessing.Pool(processes=5)
pool.map(run, files)